In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time

In [2]:
import cifar_pre_process
dt = cifar_pre_process.CifarPreProcess()
dt.set_up_images()

Setting Up Training Images and Labels
Setting Up Test Images and Labels


# Import network, layers and functions

In [3]:
from network import Sequential,layers
from layers import conv2d,max_pool,flatten,dense
import functions
import numpy as np

# Make Sequential Model

In [4]:
model=Sequential()

In [5]:
model.add(conv2d(input_shape=(32,32,3),num_kernels=32,kernel_size=3,activation=functions.relu))
model.add(conv2d(num_kernels=32,kernel_size=3,activation=functions.relu))
model.add(max_pool())
# model.add(conv2d(num_kernels=32,kernel_size=3,activation=functions.relu))
# model.add(conv2d(num_kernels=32,kernel_size=3,activation=functions.relu))
# model.add(max_pool())
model.add(conv2d(num_kernels=64,kernel_size=3,activation=functions.relu))
model.add(conv2d(num_kernels=64,kernel_size=3,activation=functions.relu))
model.add(max_pool())
model.add(flatten())
model.add(dense(512,activation=functions.relu))
model.add(dense(10,activation=functions.softmax))

# View Model Summary

In [6]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Name (type)                     Output Shape             Activation        Param #
input_layer (InputLayer)       (None, 32, 32, 3)          echo             0
__________________________________________________________________________________________
conv2d (conv2d)                (None, 32, 32, 32)         relu             896
__________________________________________________________________________________________
conv2d (conv2d)                (None, 32, 32, 32)         relu             9248
__________________________________________________________________________________________
max_pool (max_pool)            (None, 16, 16, 32)         echo             0
__________________________________________________________________________________________
conv2d (conv2d)                (None, 16, 16, 64)         relu             18496
____________________________________________________________________

# Compile model with optimizer, loss and lr

In [7]:
model.compile(optimizer=functions.adam,loss=functions.cross_entropy_with_logits,learning_rate=0.001)

# Train model

In [8]:
losses=[]
train_acc=[]

In [9]:
def validate():
    print("Calculating Validation acc...",end='')
    vdsz=512
#     ck=np.random.randint(0,10000-vdsz)
    ck=6000         # 44.25 %
    inp = dt.test_images[ck:ck+vdsz].reshape(-1,32,32,3)
    y_inp = dt.test_labels[ck:ck+vdsz]
    logits=model.predict(inp)
    ans=logits.argmax(axis=1)
    cor=y_inp.argmax(axis=1)
    acc=100*(ans==cor).mean()
    cross_entropy_loss=model.loss(logits,labels=y_inp).mean()*10
    print("\rValidation Acc: {:.3f} %        Val loss: {:.8f}".format(acc,cross_entropy_loss))

In [10]:
def run():
    st=0
    btsz=8
    acc_tn=0
    _cc=0
    loss_tn=0
    while st<=50000:
        sam_tm=time()
        perc=st/500
        ck=np.random.randint(0,50000-btsz)
        inp,y_inp=dt.batch_gen(ck=ck,size=btsz)
        logits=model.fit(inp,y_inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc=100*(ans==cor).mean()
        cross_entropy_loss=model.loss(logits=logits,labels=y_inp).mean()*10
        acc_tn+=acc
        _cc+=1
        loss_tn+=cross_entropy_loss
        acc=acc_tn/_cc
        loss_=loss_tn/_cc
        losses.append(loss_)
        train_acc.append(acc)
        if acc>=68:
            model.learning_rate=1e-5
        elif acc>=57:
            model.learning_rate=1e-5
        elif acc>=47:
            model.learning_rate=1e-4
        sam_tm=time()-sam_tm
        rem_sam=(50000-st)/btsz
        eta=int(rem_sam*sam_tm)
        print("\rProgress: {:.2f} %    Acc: {:.3f} %    loss: {:.6f}     Sample time: {:.3f}s    ETA: {}:{}s    .".format(perc,acc,loss_,sam_tm,eta//60,eta%60),end='')
        st+=btsz
    print("")

In [12]:
epochs=2
for epoch in range(epochs):
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run()
    print("Epoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))
    validate()

EPOCH: 1 / 2
Progress: 1.84 %    Acc: 20.151 %    loss: 2.121495     Sample time: 0.175s    ETA: 17:51s    .

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(losses,color='#ff7f0e')
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(train_acc,color='#ff7f0e')
plt.show()

In [ ]:
ck=np.random.randint(0,50000)
inp,y_inp=dt.batch_gen(size=1,ck=ck)
logits=model.predict(inp)
ans=logits.argmax(axis=1)
cor=y_inp.argmax(axis=1)
logits=logits[0]
print("Prediction: ",dt.names[int(ans[0])],"\nCorrect answer: ",dt.names[int(cor[0])])
plt.imshow(inp[0],interpolation='bicubic')
plt.show()
plt.barh(range(len(logits)),logits,color='#ff7f0e')
plt.yticks(range(len(logits)),dt.names)
plt.xticks([])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=5,ncols=5,figsize=[10,10])
fig.patch.set_facecolor('white')
pme=model.sequence[3].z_out[0].transpose(2,0,1)
for i,axi in enumerate(ax.flat):
    axi.imshow(pme[i].reshape(pme.shape[-2:]), cmap='Greys')